<a href="https://colab.research.google.com/github/Janina712/MLTSA22_JBoecher/blob/main/Other/3.%20Match_MLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. Import & Set-Up**

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import os
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/2.BreathGroups_Assigned/

/content/gdrive/MyDrive/RhythmAnalysisPipeline/2.BreathGroups_Assigned


In [4]:
reading = pd.read_excel("reading_TextGrid_comb_BG_loop.xlsx")
interview = pd.read_excel("interview_TextGrid_comb_BG_loop.xlsx")

In [5]:
IDs = ['24fa', '24ma', '24mb', '26f', '29ma', '29mb', '29mc', '32m', '34m', '35mb', '46ma', '50fa', '50fb', '54f', '57m', '60m', '62f', '62m','C1_DS', 'C2_JH', 'C3_MD', 'C4_ZO', 'C5_HL', 'C6_LH', 'C7_SH', 'C9_RD',  'C10_KS', 'C11_NP', 'C12_CC', 'C13_RG']

In [6]:
subj_col = pd.DataFrame(['24fa', '24ma', '24mb', '26f', '29ma', '29mb', '29mc', '32m', '34m', '35mb', '46ma', '50fa', '50fb', '54f', '57m', '60m', '62f', '62m','C1_DS', 'C2_JH', 'C3_MD', 'C4_ZO', 'C5_HL', 'C6_LH', 'C7_SH', 'C9_RD', 'C10_KS', 'C11_NP', 'C12_CC', 'C13_RG'], columns = ["ID"])

In [7]:
control_IDs = ['C1_DS', 'C2_JH', 'C3_MD', 'C4_ZO', 'C5_HL', 'C6_LH', 'C7_SH', 'C9_RD', 'C10_KS', 'C11_NP', 'C12_CC', 'C13_RG']

# **Define Functions**

**1. Assign Participant Group**

In [8]:
def assign_group(df):
  group = pd.DataFrame(index = range(len(df)),columns=["Group"])

  for i in range(0,len(df)):
    if "_" in df["ID"][i]:
      group["Group"][i] = "Control"
    else:
      group["Group"][i] = "PWS"
  
  df_out = pd.concat([ group, df], axis=1)
  df_out = df_out[df_out.Type != "silence"]
  df_out.index = range(len(df_out.index))
  df_out.drop(['Unnamed: 0'], axis=1 , inplace = True)
  return(df_out)

**2. Count Vowels**

In [9]:
def count_vowels(df):  
  df_vowels = df[df["Type"]  == "vowel"] 
  df_vowels.index = range(len(df_vowels.index))

  syll_col = pd.DataFrame()  ## initialize group-level dataframe
  for ID in IDs: ## loop over participnts
    syll_current_ID = pd.DataFrame()   ## initialize participant-level dataframe
    subset_sounds = df[df["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
    subset_sounds.index = range(len(subset_sounds.index)) # reset index
    subset_vowels = subset_sounds[subset_sounds["Type"] == "vowel"]  # get subset of sound dataframe that corresponds to current participant
    subset_vowels.index = range(len(subset_vowels.index)) # reset index
    syll = subset_vowels["Breath.Group"].value_counts().sort_index() # count how often a certain Breath group occurs for this participant
    syll.index = range(len(syll.index)) # reset index
    for a in range (0,len(syll)): # go through all breath groups that this participant produced
      syll_current_BG = pd.DataFrame()  ## initialize BG-level dataframe
      syll_current_BG = pd.DataFrame(np.repeat(syll.iloc[a], syll.iloc[a], axis=0)) #replicate the sum sum times
      syll_current_ID = syll_current_ID.append([syll_current_BG], ignore_index = True) # add BG-level dataframe to participant-level dataframe 
    syll_col = syll_col.append([syll_current_ID], ignore_index = True) # add participant-level dataframe to group-level dataframe

  df_vowels = pd.concat([df_vowels, syll_col], axis=1)
  #df_vowels.drop(['Syllables'], axis=1 , inplace = True)
  df_vowels.rename(columns = {'Syllables':'Unmached_Vowels'}, inplace = True)
  df_vowels.rename(columns = {0:'Syllables'}, inplace = True) # rename new column
  pre_df_vowel_avg = df_vowels.groupby("Group").mean()    ########### average counting 13 13 times

  return(df_vowels, pre_df_vowel_avg)

**3. Count Consonants**

In [10]:
def count_consonants(df):  
  df_consonants = df[df["Type"]  == "consonant"] 
  df_consonants.index = range(len(df_consonants.index))

  con_col = pd.DataFrame()  ## initialize group-level dataframe
  for ID in IDs: ## loop over participnts
    con_current_ID = pd.DataFrame()   ## initialize participant-level dataframe
    subset_sounds = df[df["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
    subset_sounds.index = range(len(subset_sounds.index)) # reset index
    subset_cons = subset_sounds[subset_sounds["Type"] == "consonant"]  # get subset of sound dataframe that corresponds to current participant
    subset_cons.index = range(len(subset_cons.index)) # reset index
    con = subset_cons["Breath.Group"].value_counts().sort_index() # count how often a certain Breath group occurs for this participant
    con.index = range(len(con.index)) # reset index
    for a in range (0,len(con)): # go through all breath groups that this participant produced
      con_current_BG = pd.DataFrame()  ## initialize BG-level dataframe
      con_current_BG = pd.DataFrame(np.repeat(con.iloc[a], con.iloc[a], axis=0)) #replicate the sum sum times
      con_current_ID = con_current_ID.append([con_current_BG], ignore_index = True) # add BG-level dataframe to participant-level dataframe 
    con_col = con_col.append([con_current_ID], ignore_index = True) # add participant-level dataframe to group-level dataframe

  df_consonants = pd.concat([df_consonants, con_col], axis=1)
  #df_consonants.drop(['Consonants'], axis=1 , inplace = True)
  df_consonants.rename(columns = {'Consonants':'Unmatched_Cons'}, inplace = True)
  df_consonants.rename(columns = {0:'Consonants'}, inplace = True) # rename new column
  pre_df_consonant_avg = df_consonants.groupby("Group").mean()    ########### average counting 13 13 times

  return(df_consonants, pre_df_consonant_avg)

**4. Average Vowel Count by Participant**

In [11]:
def participant_vowel_avg(df):  
  group_col = pd.DataFrame(index = range(len(subj_col)),columns=["Group"])
  for i in range(0,len(group_col)):
    if "_" in subj_col["ID"][i]:
      group_col["Group"][i] = "Control"
    else:
      group_col["Group"][i] = "PWS"
  
  n = -1
  avg_col = pd.DataFrame(index = range(len(subj_col)),columns=["Syllables"])
  for ID in IDs: ## loop over participnts
    n = n + 1
    subset_BGs = df[df["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
    subset_BGs.index = range(len(subset_BGs.index)) # reset index
    BG_avg = subset_BGs.groupby("Breath.Group").mean()
    subj_avg = BG_avg["Syllables"].mean()
    avg_col["Syllables"][n] = subj_avg

  df_participant_vowel_avg = pd.concat([group_col, subj_col, avg_col], axis=1)

  return(df_participant_vowel_avg)

**5. Average Consonant Count by Participant**

In [12]:
def participant_consonant_avg(df):  
  group_col = pd.DataFrame(index = range(len(subj_col)),columns=["Group"])
  for i in range(0,len(group_col)):
    if "_" in subj_col["ID"][i]:
      group_col["Group"][i] = "Control"
    else:
      group_col["Group"][i] = "PWS"
  
  n = -1
  avg_col = pd.DataFrame(index = range(len(subj_col)),columns=["Consonants"])
  for ID in IDs: ## loop over participnts
    n = n + 1
    subset_BGs = df[df["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
    subset_BGs.index = range(len(subset_BGs.index)) # reset index
    BG_avg = subset_BGs.groupby("Breath.Group").mean()
    subj_avg = BG_avg["Consonants"].mean()
    avg_col["Consonants"][n] = subj_avg

  df_participant_cons_avg = pd.concat([group_col, subj_col, avg_col], axis=1)

  return(df_participant_cons_avg)

**6. Compare Consonant and Vowel Counts Across Participant Groups (PWS vs. Control)**

In [13]:
def compare_groups(df_vowels, df_consonants):
  #comparison_v = df_vowels.groupby("Group").mean()
  #control_v = comparison_v["Syllables"][0]
  #pws_v = comparison_v["Syllables"][1]
  control_v = (df_vowels.groupby("ID").mean()["Syllables"][18:30]).mean()
  pws_v = (df_vowels.groupby("ID").mean()["Syllables"][0:18]).mean()
  difference_v = control_v - pws_v

  #comparison_c = df_consonants.groupby("Group").mean()
  #control_c = comparison_c["Consonants"][0]
  #pws_c = comparison_c["Consonants"][1]
  control_c = (df_consonants.groupby("ID").mean()["Consonants"][18:30]).mean()
  pws_c = (df_consonants.groupby("ID").mean()["Consonants"][0:18]).mean()
  difference_c = control_c - pws_c

  string1 = (f"PWS produced on average {round(pws_v,2)} syllables per utterance, while control participants produced {round(control_v,2)} syllables on average.")
  string2 = (f"This means that on average control participants produced {round(difference_v,2)} syllables more per utterance.")
  string3 = (f"\nPWS produced on average {round(pws_c,2)} consonants per utterance, while control participants produced {round(control_c,2)} consonants on average.")
  string4 = (f"This means that on average control participants produced {round(difference_c,2)} consonants more per utterance.")

  return(string1, string2, string3, string4, pws_v, pws_c, difference_v, difference_c)

**7. Match Number of Vowels Per Utterance**

In [14]:
def match_vowels(df_vowels, pws_v, cut_v):
  cut_v = math.ceil(cut_v) ##+ 5                                                           ################## number chosen because "it works well". 
  syll_col_matched = pd.DataFrame()  ## initialize group-level dataframe              ################ No real motivation. Come up with strategy! ############
  participant = pd.DataFrame()  ## initialize participant-level dataframe
  df_control = df_vowels[df_vowels["Group"]  == "Control"] 
  df_control.index = range(len(df_control.index)) ## group
  for ID in control_IDs:
    df_control_ID = df_control[df_control["ID"]  == ID] 
    df_control_ID.index = range(len(df_control_ID.index))  ### person
    BGs = df_control_ID["Breath.Group"].unique()
    for i in range(0,len(BGs)):
      df_control_ID_BG = df_control_ID[df_control_ID["Breath.Group"]  == i] 
      df_control_ID_BG.index = range(len(df_control_ID_BG.index)) ## BG
      if len(df_control_ID_BG) >= cut_v:
        df_control_ID_BG.drop(df_control_ID_BG.tail(cut_v).index, inplace = True) 
        participant = participant.append([df_control_ID_BG], ignore_index = True)
      else:  
        participant = participant.append([df_control_ID_BG], ignore_index = True)
  syll_col_matched = syll_col_matched.append([participant], ignore_index = True) # add participant-level dataframe to group-level dataframe

  return(syll_col_matched)

**8. Match Number of Consonants Per Utterance**

In [15]:
def match_consonants(df_consonants, pws_c, cut_c):
  cut_c = math.ceil(cut_c) ##+ 4                                                       ################## number chosen because "it works well".
  cons_col_matched = pd.DataFrame()  ## initialize group-level dataframe          ################ No real motivation. Come up with strategy! ############
  participant = pd.DataFrame()  ## initialize participant-level dataframe
  df_control = df_consonants[df_consonants["Group"]  == "Control"] 
  df_control.index = range(len(df_control.index)) ## group
  for ID in control_IDs:
    df_control_ID = df_control[df_control["ID"]  == ID] 
    df_control_ID.index = range(len(df_control_ID.index))  ### person
    BGs = df_control_ID["Breath.Group"].unique()
    for i in range(0,len(BGs)):
      df_control_ID_BG = df_control_ID[df_control_ID["Breath.Group"]  == i] 
      df_control_ID_BG.index = range(len(df_control_ID_BG.index)) ## BG
      if len(df_control_ID_BG) >= cut_c:
        df_control_ID_BG.drop(df_control_ID_BG.tail(cut_c).index, inplace = True) 
        participant = participant.append([df_control_ID_BG], ignore_index = True)
      else:  
        participant = participant.append([df_control_ID_BG], ignore_index = True)
  cons_col_matched = cons_col_matched.append([participant], ignore_index = True) # add participant-level dataframe to group-level dataframe
  return(cons_col_matched)

**9. Test if Remaining Vowel Difference Between Groups is Statistically Significant**

In [16]:
def ttest_vowelDifference(matched_participant_df, cut_v): 
  cut_v = round(cut_v)
  ttest_control = matched_participant_df[18:30]  
  ttest_pws = matched_participant_df[0:18]   

  control_v =  matched_participant_df[18:30].mean()[0]
  pws_v =  matched_participant_df[0:18].mean()[0]

  difference_v_matched = control_v - pws_v

  df_v_matched_test = stats.ttest_ind(ttest_control['Syllables'],
                        ttest_pws['Syllables'])
  if df_v_matched_test[1] >= 0.05:
    level = "insignificant"
  else:
    level = "significant"
  string11 = (f"Control utterances that were longer than the mean average length of PWS utterances were shortened. \nAfter matching, PWS had {round(pws_v,2)} vowels per utterance, while control participants had {round(control_v,2)} per utterance. \nThe difference is reduced to {abs(round(difference_v_matched,2))}, which is statistically {level} (p = {round(df_v_matched_test[1],2)}).")

  return(string11)

**10. Test if Remaining Consonant Difference Between Groups is Statistically Significant**

In [17]:
def ttest_conDifference(matched_participant_df, cut_c):  
  cut_c = round(cut_c)
  ttest_control = matched_participant_df[18:30]
  ttest_pws = matched_participant_df[0:18]  

  control_c =  matched_participant_df[18:30].mean()[0]
  pws_c =  matched_participant_df[0:18].mean()[0]

  difference_c_matched = control_c - pws_c

  df_c_matched_test = stats.ttest_ind(ttest_control['Consonants'],
                        ttest_pws['Consonants'])
  if df_c_matched_test[1] >= 0.05:
    level = "insignificant"
  else:
    level = "significant"
  string11 = (f"Control utterances that were longer than the mean average length of PWS utterances, were shortened by {round(cut_c,2)} consonants. \nAfter matching, PWS had {round(pws_c,2)} consonants per utterance, while control participants had {round(control_c,2)} per utterance. \nThe difference is reduced to {abs(round(difference_c_matched,2))}, which is statistically {level} (p = {round(df_c_matched_test[1],2)}).")

  return(string11)

# **Conduct Matching Analysis using Functions**

In [18]:
# add new column to dataframe that denotes participant's group membership (PWS vs. Control)
interview = assign_group(interview)
reading = assign_group(reading)

In [19]:
# count vowels per utterance 
# output 1: df of size "interview", with an additional #vowels per utterance for each row (redundant for BG)
# output 2: df of size "len(Group)" (2), that contains average # vowels per breath group for PWS group vs. Control group 
[interview_vowels, pre_interview_vowel_avg]  = count_vowels(interview)
[reading_vowels, pre_reading_vowel_avg] = count_vowels(reading)

In [20]:
# count consonants per utterance 
# output 1: df of size "interview", with an additional #consonants per utterance for each row (redundant for BG)
# output 2: df of size "len(Group)" (2), that contains average # consonants per breath group for PWS group vs. Control group 
[interview_consonants, pre_interview_consonant_avg]  = count_consonants(interview)
[reading_consonants, pre_reading_consonants_avg] = count_consonants(reading)

In [21]:
#average # vowels per breath group for each participant 
interview_participant_vowel_avg = participant_vowel_avg(interview_vowels)
reading_participant_vowel_avg = participant_vowel_avg(reading_vowels)

In [22]:
#average # consonants per breath group for each participant 
interview_participant_cons_avg = participant_consonant_avg(interview_consonants)
reading_participant_cons_avg = participant_consonant_avg(reading_consonants)

In [23]:
# compare group averages of number of vowels per utterance and number of consonants per utterance 
# interview
string1, string2, string3, string4, pws_v_inter, pws_c_inter, difference_v_inter, difference_c_inter = compare_groups(interview_participant_vowel_avg, interview_participant_cons_avg)

print(string1)
print(string2)
print(string3)
print(string4)

PWS produced on average 7.87 syllables per utterance, while control participants produced 9.84 syllables on average.
This means that on average control participants produced 1.96 syllables more per utterance.

PWS produced on average 11.59 consonants per utterance, while control participants produced 15.38 consonants on average.
This means that on average control participants produced 3.79 consonants more per utterance.


In [24]:
# compare group averages of number of vowels per utterance and number of consonants per utterance 
# reading
string5, string6, string7, string8, pws_v_read, pws_c_read, difference_v_read, difference_c_read  = compare_groups(reading_participant_vowel_avg, reading_participant_cons_avg)

print(string5)
print(string6)
print(string7)
print(string8)

PWS produced on average 8.54 syllables per utterance, while control participants produced 11.76 syllables on average.
This means that on average control participants produced 3.22 syllables more per utterance.

PWS produced on average 13.13 consonants per utterance, while control participants produced 18.05 consonants on average.
This means that on average control participants produced 4.93 consonants more per utterance.


In [25]:
# match number of vowels per utterance by cutting long control utterances (longer than pws average) by the difference in average vowel number between groups
matched_vowels_read = match_vowels(reading_vowels,pws_v_read, difference_v_read)
matched_vowels_inter = match_vowels(interview_vowels, pws_v_inter, difference_v_inter)

In [26]:
# match number of consonants per utterance by cutting long control utterances (longer than pws average) by the difference in average consonant number between groups
matched_consonants_read = match_vowels(reading_consonants,pws_c_read, difference_c_read)
matched_consonants_inter = match_vowels(interview_consonants, pws_c_inter, difference_c_inter)

In [27]:
# count vowels per utterance after matching
# output 1: df of size "interview minus cut vowel rows", with an updated #vowels per utterance for each row (redundant for BG)
# output 2: df of size "len(Group)" (2), that contains average # vowels per breath group for PWS group vs. Control group after matching
[post_interview_vowels, post_interview_vowel_avg]  = count_vowels(matched_vowels_inter)
[post_reading_vowels, post_reading_vowel_avg]  = count_vowels(matched_vowels_read)

In [28]:
# average # vowels per breath group for each participant after matching
post_interview_participant_vowels_avg = participant_vowel_avg(post_interview_vowels)
post_reading_participant_vowels_avg = participant_vowel_avg(post_reading_vowels)
# only control group has been changed
post_interview_participant_vowels_avg = post_interview_participant_vowels_avg[18:30]
post_reading_participant_vowels_avg = post_reading_participant_vowels_avg[18:30]
# combine with pre_matching (unchanged pws)avgs
post_interview_participant_vowels_avg = interview_participant_vowel_avg[0:18].append(post_interview_participant_vowels_avg)
post_reading_participant_vowels_avg = reading_participant_vowel_avg[0:18].append(post_reading_participant_vowels_avg)

In [29]:
# count consonants per utterance after matching
# output 1: df of size "interview minus cut consonant rows", with an updated #consonants per utterance for each row (redundant for BG)
# output 2: df of size "len(Group)" (2), that contains average # consonants per breath group for PWS group vs. Control group after matching 
[post_interview_consonants, post_interview_consonant_avg]  = count_consonants(matched_consonants_inter)
[post_reading_consonants, post_reading_consonant_avg]  = count_consonants(matched_consonants_read)

In [30]:
#average # consonants per breath group for each participant after matching
post_interview_participant_cons_avg = participant_consonant_avg(post_interview_consonants)
post_reading_participant_cons_avg = participant_consonant_avg(post_reading_consonants)
# only control group has been changed
post_interview_participant_cons_avg = post_interview_participant_cons_avg[18:30]
post_reading_participant_cons_avg = post_reading_participant_cons_avg[18:30]
# combine with pre_matching (unchanged pws)avgs
post_interview_participant_cons_avg = interview_participant_cons_avg[0:18].append(post_interview_participant_cons_avg)
post_reading_participant_cons_avg = reading_participant_cons_avg[0:18].append(post_reading_participant_cons_avg)

In [31]:
# compare group averages of number of vowels per utterance and number of consonants per utterance
[string1, string2, string3, string4, pws_v_inter, pws_c_inter, difference_v_inter, difference_c_inter] = compare_groups(post_interview_participant_vowels_avg, post_interview_participant_cons_avg)

print(string1)
print(string2)
print(string3)
print(string4)

PWS produced on average 7.87 syllables per utterance, while control participants produced 7.82 syllables on average.
This means that on average control participants produced -0.05 syllables more per utterance.

PWS produced on average 11.59 consonants per utterance, while control participants produced 11.44 consonants on average.
This means that on average control participants produced -0.15 consonants more per utterance.


In [32]:
[string5, string6, string7, string8, control_v_read, pws_v_read, difference_v_read, difference_c_read] = compare_groups(post_reading_participant_vowels_avg, post_reading_participant_cons_avg)
print(string5)
print(string6)
print(string7)
print(string8)

PWS produced on average 8.54 syllables per utterance, while control participants produced 8.37 syllables on average.
This means that on average control participants produced -0.17 syllables more per utterance.

PWS produced on average 13.13 consonants per utterance, while control participants produced 13.43 consonants on average.
This means that on average control participants produced 0.31 consonants more per utterance.


In [33]:
# test if average number of vowels per breathgroup is still significantly different between groups after matching
string1 = ttest_vowelDifference(post_interview_participant_vowels_avg, difference_v_inter)
print(string1)


Control utterances that were longer than the mean average length of PWS utterances were shortened. 
After matching, PWS had 7.87 vowels per utterance, while control participants had 7.82 per utterance. 
The difference is reduced to 0.05, which is statistically insignificant (p = 0.93).


In [34]:
string2 = ttest_vowelDifference(post_reading_participant_vowels_avg, difference_v_read)
print(string2)

Control utterances that were longer than the mean average length of PWS utterances were shortened. 
After matching, PWS had 8.54 vowels per utterance, while control participants had 8.37 per utterance. 
The difference is reduced to 0.17, which is statistically insignificant (p = 0.73).


In [35]:
# test if average number of consonants per breathgroup is still significantly different between groups after matching
string1 = ttest_conDifference(post_interview_participant_cons_avg, difference_c_inter)
print(string1)

Control utterances that were longer than the mean average length of PWS utterances, were shortened by 0 consonants. 
After matching, PWS had 11.59 consonants per utterance, while control participants had 11.44 per utterance. 
The difference is reduced to 0.15, which is statistically insignificant (p = 0.87).


In [36]:
string2 = ttest_conDifference(post_reading_participant_cons_avg, difference_c_read)
print(string2)

Control utterances that were longer than the mean average length of PWS utterances, were shortened by 0 consonants. 
After matching, PWS had 13.13 consonants per utterance, while control participants had 13.43 per utterance. 
The difference is reduced to 0.31, which is statistically insignificant (p = 0.68).


# **Export Results**

**1. Create Output DataFrames**

In [37]:
# reading vowels
pws = reading_vowels[reading_vowels["Group"] == "PWS"]  # get subset of sound dataframe that corresponds to current participant
post_reading_vowels.drop(["Unmached_Vowels"],axis=1 , inplace = True)
reading_vowels_matched = pws.append(post_reading_vowels)

In [38]:
# interview vowels
pws = interview_vowels[interview_vowels["Group"] == "PWS"]  # get subset of sound dataframe that corresponds to current participant
post_interview_vowels.drop(["Unmached_Vowels"],axis=1 , inplace = True)
interview_vowels_matched = pws.append(post_interview_vowels)

In [39]:
# reading consonants
pws = reading_consonants[reading_consonants["Group"] == "PWS"]  # get subset of sound dataframe that corresponds to current participant
post_reading_consonants.drop(["Unmatched_Cons"],axis=1 , inplace = True)
reading_consonants_matched = pws.append(post_reading_consonants)

In [40]:
# interview consonants
pws = interview_consonants[interview_consonants["Group"] == "PWS"]  # get subset of sound dataframe that corresponds to current participant
post_interview_consonants.drop(["Unmatched_Cons"],axis=1 , inplace = True)
interview_consonants_matched = pws.append(post_interview_consonants)

**2. Create Directory and Save**

In [41]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

/content/gdrive/MyDrive/RhythmAnalysisPipeline


In [42]:
dir = "3.MLU_Matched"

if os.path.exists(dir) == False:
  os.mkdir(dir)

In [43]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/3.MLU_Matched/

/content/gdrive/MyDrive/RhythmAnalysisPipeline/3.MLU_Matched


In [44]:
interview_vowels_matched.to_excel("interview_TextGrid_comb_BG_vowelsMatched.xlsx")
interview_consonants_matched.to_excel("interview_TextGrid_comb_BG_consonantsMatched.xlsx")
reading_vowels_matched.to_excel("reading_TextGrid_comb_BG_vowelsMatched.xlsx")
reading_consonants_matched.to_excel("reading_TextGrid_comb_BG_consonantsMatched.xlsx")

In [45]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

/content/gdrive/MyDrive/RhythmAnalysisPipeline
